# Import Library

In [17]:
# Import Library
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
from pymongo import MongoClient
from langchain_openai import ChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import ChatOpenAI
import re

# Set Up

In [18]:
# Load semua environment
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
DEEPSEEK_API_NEW=os.getenv("DEEPSEEK_API_NEW")

In [19]:
# Embeding menggunakan gemini
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY,
)

In [20]:
# MongoDB Connection ke
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq_300']

In [21]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
)

In [22]:
# Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Virtual Resmi BPJS Indonesia.

    Peran Anda:
    Anda bertindak sebagai Customer Service (Layanan Pelanggan) untuk membantu masyarakat dalam memahami layanan dan prosedur dari:
    - Website BPJS Kesehatan
    - FAQ BPJS Ketenagakerjaan
    - FAQ Aplikasi JKN Mobile

    Tugas & Tanggung Jawab:
    1. Memberikan jawaban akurat, terpercaya, dan relevan sesuai dengan data dari sumber resmi (FAQ).
    2. Menjelaskan prosedur atau solusi dengan cara yang mudah dimengerti, terstruktur, dan menggunakan Bahasa Indonesia baku.
    3. Menjadi perantara informasi yang ramah, efisien, dan profesional dalam menangani keluhan atau pertanyaan pengguna.
    4. Jawablah dengan ringkas, jangan menjawab hal yang tidak ditanyakan
    5. Pertanyaan mungkin menggunakan bahasa tidak baku atau typo, sehingga anda harus paham perbedaannya dengan konteks yang berada diluar FAQ.
    7. Periksalah apakah pertanyaan merupakan suatu singkatan, yang mungkin terdapat jawabannya di dataset.

    Aturan yang Wajib Dipatuhi:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam dokumen FAQ resmi.
    2. Jangan mengarang jawaban atau memberikan informasi tambahan yang tidak tercantum dalam sumber resmi.
    3. Jika pertanyaan tidak relevan atau tidak tersedia dalam data, sampaikan secara sopan bahwa Anda tidak memiliki informasi tersebut.
    4. Dilarang mencantumkan tautan eksternal, nomor telepon, alamat, atau informasi pribadi lain yang tidak ada dalam dokumen FAQ, jika ada diperbolehkan.
    5. Gunakan:
    - Penomoran (1., 2., 3., dst) untuk menjelaskan langkah atau prosedur berurutan.
    - Simbol bullet (•) untuk daftar yang tidak berurutan.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [23]:
# Config model untuk Deepseek
llm_deepseek = ChatOpenAI(
    model="deepseek-chat",  
    openai_api_key=DEEPSEEK_API_NEW,  
    base_url="https://api.deepseek.com/v1",  
    temperature=0,
    max_tokens=500
)

In [24]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_deepseek,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.70
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
)

In [25]:
# Ambil data
faq_df = pd.read_csv('../data/final_dataset_clean.csv')
def answer_from_doc(question):
    """Fungsi untuk mencocokkan pertanyaan dengan data yang sama persis dan mengambil jawaban"""
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

In [26]:
def clean_answer(text):
    """Hilangkan tanda bold/italic Markdown"""
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    cleaned_text = re.sub(r'\n{3,}', '\n\n', text)
    return cleaned_text.strip()
    

# Evaluation

## Test Case 1 Pertanyaan Dataset

In [27]:
# UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset
questions = [
# Mobile
"Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?",
"Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
"Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?",
"Apakah itu Promo?",
"Apakah itu MotionPay?",
"Apakah sandi/PIN harus diubah secara berkala di kedua aplikasi?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",

# Ketenagakerjaan
"Apa itu BPJS Ketenagakerjaan?",
"Apa saja program BPJS Ketenagakerjaan?",
"Siapakah yang dimaksud peserta BPJS Ketenagakerjaan?",
"Apakah yang dimaksud dengan PHK?",
"Apakah pegawai BHL (Buruh Harian Lepas) dapat mengikuti program JKP?",
"Bagaimana caranya badan usaha/pemberi kerja dapat menggunakan layanan Payment Reminder System(PRS)?",
"Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?",
"Apakah aplikasi Electronic Payment System (EPS)?",
"Apa perbedaan program Jaminan Hari Tua (JHT) dan Jaminan Pensiun (JP)?",

# Kesehatan
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Bagaimana Mendaftarkan Bayi Baru Lahir ?",
"Dimana kontak kantor BPJS Kesehatan ?",
"Apa itu GRC ?",
"Cara Membayar Iuran",
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Kapan pembayaran iuran paling lambat ?",
"Bagaimana komitmen BPJS Kesehatan terkait gratifikasi ?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?
Untuk menginstal aplikasi Jamsostek Mobile (JMO), berikut langkah-langkahnya:  

1. Buka Google Play Store (Android) atau App Store (iOS) di perangkat Anda.  
2. Cari "Jamsostek Mobile" di kolom pencarian.  
3. Pilih aplikasi resmi yang diterbitkan oleh BPJS Ketenagakerjaan.  
4. Klik "Instal" dan tunggu proses pengunduhan selesai.  
5. Buka aplikasi dan ikuti panduan registrasi/login sesuai kebutuhan.  

Pastikan perangkat Anda terhubung dengan internet dan memiliki ruang penyimpanan yang cukup.  

Jika mengalami kendala, pastikan aplikasi yang diunduh adalah versi terbaru dan sesuai dengan sistem operasi perangkat Anda.

ACTUAL DATA: Aplikasi Jamsostek Mobile (JMO) dapat diunduh dan diinstall melalui Playstore/ Appstore bagi pengguna smartphone .

Pertanyaan: Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?
Untuk mengajukan klaim Jaminan Hari Tua (JHT) melalui JMO (Jamsostek Mobi

Total pertanyaan sesuai dataset : 25

Pertanyaan terjawab             : 16

Pertanyaan tidak terjawab       : 9

Pertanyaan tidak terjawab:
1. Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?
2. Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?
3. Apakah itu Promo?
4. Apakah itu MotionPay?
5. Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?
6. Apakah aplikasi Electronic Payment System (EPS)?
7. Dimana kontak kantor BPJS Kesehatan ?
8. Apa itu GRC ?
9. Bagaimana komitmen BPJS Kesehatan terkait gratifikasi ?

Pertanyaan yang tidak dapat di jawab mayoritas adalah pertanyaan dari Ketenagakerjaan dengan category E-CHANNEL, dan sub-kategori JMO. 

Beberapa asumsi kenapa pertanyaan tersebut tidak dapat di jawab:
1. Pertanyaan tidak begitu jelas seperti promo, motionplay yang diartikan oleh bot sebagai di luar konteks BPJS, sehingga Deepseek tidak memberikan informasi, akan di coba improve pada promptnya.
2. Size embeding 300 dan overlap 50 kemungkinan terlalu kecil sehingga kurang membuat model mengerti reasoning pertanyaan dan jawaban.
3. Treshold similarity sebesar 70%, akan di coba untuk di tingkatkan.

## Test Case 2 Pertanyaan Diluar Dataset

In [28]:
# UJI COBA 2: Pertanyaan di Luar Konteks
questions = [
    "Apa itu Allo Bank?",
    "Bagaimana cara mengajukan KPR?",
    "Siapa penemu listrik?",
    "Apa saja manfaat dari olahraga?",
    "Bagaimana cara membuat kue?",
    "Jelaskan apa itu Quantum Computing?",
    "Apa yang dimaksud dengan Machine Learning?",
    "Gimana cara buat foto HD dengan AI?",
    "Apa itu teknologi blockchain?",
    "Apa itu teknologi quantum?",
    "Apa kopi paling enak saat ini?",
    "Sebutkan Sepatu lari paling enak",
    "Kapan Indonesia Merdeka?",
    "Dimana kantor urusan agama terdekat",
    "Jelaskan tentang skema piala dunia?",
    "Siapa pencetak gol terbanyak piala dunia 2022?",
    "Jumlah total trofi Liverpool berapa?",
    "Hitung luas Kota Surabaya",
    "Dimana tempat syuting film laskar Pelangi?",
    "Dimana bom atom amerika saat perang dunia kedua dibuat?",
    "Bagaimana cara membedakan biji kopi?",
    "Jembatan merah ada Dimana?",
    "Siapa presiden Indonesia pertama?",
    "Siapa penemu alat musik gitar?",
    "Hitung luas lapangan bola."
    ]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Maaf, pertanyaan Anda tentang "Allo Bank" tidak terkait dengan layanan BPJS Kesehatan, BPJS Ketenagakerjaan, atau Aplikasi JKN Mobile. Sebagai Asisten Virtual BPJS Indonesia, saya hanya dapat membantu pertanyaan yang berkaitan dengan ketiga layanan tersebut sesuai dokumen resmi.  

Jika Anda memiliki pertanyaan seputar BPJS, silakan ajukan pertanyaan yang lebih spesifik. Terima kasih.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Bagaimana cara mengajukan KPR?
Maaf, pertanyaan Anda mengenai cara mengajukan KPR (Kredit Pemilikan Rumah) tidak termasuk dalam lingkup layanan BPJS Kesehatan, BPJS Ketenagakerjaan, atau Aplikasi JKN Mobile.  

Untuk informasi terkait KPR, disarankan menghubungi pihak bank atau lembaga keuangan terkait.  

Jika Anda memiliki pertanyaan seputar layanan BPJS, saya dengan senang hati akan membantu.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Siapa penemu listrik?
Mohon maaf, saya tidak dapat memberikan informasi tersebut karena p

Total pertanyaan diluat dataset : 25

Pertanyaan terjawab             : 0

Pertanyaan tidak terjawab       : 25

Peforma model sangat baik untuk tidak menjawab pertanyaan yang tidak relevan dengan BPJS, treshold 70% dianggap sudah cukup untuk model tidak menjawab pertanyaan tidak relevan

## Test Case 3 Pertanyaan Tidak Lengkap/Typo

In [29]:
# UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan input
questions = [
   "Aph itu BPJS Ketenagakerajaan?",
    "Apa sajak program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?",
    "Apakh yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?", 
    "Gimana jika lupa PIN EPS?",
    "Apakah itu Perumahan Pekerja?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Kapan BSU tahun 2025 dibagikan?",
    "Dimana obat batuk pertama kali ditemukan",
    "Jika flu sebaiknya minum obat herbal apa",
    "Sopo peserta BPJS ketenagakerjaan",
    "Sejak tanggal berapa pembayaran iuran paling lambat ?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?",
    "Apakah itu ShoopePay?",
    "Apa perbedaan program Jaminan Dari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Gimana pembayaran iuran?",
    "Tanggal pembayaran iuran paling lambat ?",
    "Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Aph itu BPJS Ketenagakerajaan?
BPJS Ketenagakerjaan adalah program perlindungan sosial bagi tenaga kerja yang diselenggarakan oleh Badan Penyelenggara Jaminan Sosial (BPJS) Ketenagakerjaan. Program ini memberikan manfaat berupa:  

• Jaminan Kecelakaan Kerja (JKK)  
• Jaminan Hari Tua (JHT)  
• Jaminan Pensiun (JP)  
• Jaminan Kematian (JKm)  

Informasi lebih lanjut dapat dilihat di situs resmi BPJS Ketenagakerjaan.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Apa sajak program BPJS Ketenagakerjaan?
Berikut adalah program-program BPJS Ketenagakerjaan berdasarkan informasi resmi:  

1. Jaminan Kecelakaan Kerja (JKK)  
   • Memberikan perlindungan atas risiko kecelakaan kerja atau penyakit akibat kerja.  

2. Jaminan Kematian (JKM)  
   • Memberikan manfaat kepada ahli waris jika peserta meninggal dunia (bukan akibat kecelakaan kerja).  

3. Jaminan Hari Tua (JHT)  
   • Memberikan manfaat berupa tabungan yang dapat dicairkan saat peserta memenuhi syarat (misalnya pensiun 

Total pertanyaan salah tidak baku/typo : 20

Pertanyaan terjawab             : 11

Pertanyaan tidak terjawab       : 9

Pertanyaan yang tidak terjawab :
1. Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?
2. Apakah itu Perumahan Pekerja?
3. Kapan BSU tahun 2025 dibagikan?
4. Dimana obat batuk pertama kali ditemukan
5. Jika flu sebaiknya minum obat herbal apa
6. Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?
7. Apakah itu ShoopePay?
8. Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?
9. Sopo peserta BPJS ketenagakerjaan

Analisa:
1. Model dapat mengerti apabila terdapat kata yang tidak lengkap atau typo namun kesalahan kata tersebut tidak merujuk pada kata yang memiliki makna lain, seperti pertanyaan Aph itu BPJS Ketenagakerajaan?, typo Aph sangat dekat dengan Apa sehingga model mampu menjawab.
2. Pertanyaan yang memiliki makna berbeda seperti Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?, dianggap model berada diluar konteks sehingga model tidak menjawab.